# Fast Neural Style Transfer - CSE 311 AI Project

**Project Title:** Real-Time Artistic Image Stylization Using Deep Neural Networks

This notebook demonstrates training and inference for Fast Neural Style Transfer on Google Colab.

---

## Overview

Neural Style Transfer allows us to apply the artistic style of one image to the content of another. This implementation uses:

- **Generator Network**: Feed-forward CNN with residual blocks (TransformerNet)
- **Loss Function**: VGG19-based content, style, and total variation losses
- **Dataset**: COCO 2017 (subset of ~2,000 images at 256×256)
- **Training**: 2-3 epochs optimized for Colab Free Tier

---

## 1. Setup and Installation

First, let's clone the repository and install dependencies.

In [ ]:
# Clone the repository (replace with your repo URL)
!git clone https://github.com/YOUR_USERNAME/YOUR_REPO.git
%cd YOUR_REPO

In [ ]:
# Install minimal dependencies
!pip install -q torch torchvision tqdm matplotlib Pillow numpy opencv-python

In [ ]:
# Verify GPU availability
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. Download Dataset and Style Image

Download a subset of COCO 2017 dataset and a style image.

In [ ]:
# Download COCO 2017 validation set (smaller than train set)
!wget -q http://images.cocodataset.org/zips/val2017.zip
!unzip -q val2017.zip
print("COCO dataset downloaded!")

In [ ]:
# Use existing style image from repo or download one
import os
from PIL import Image
import matplotlib.pyplot as plt

# Display the style image
style_image_path = "picasso_selfportrait.jpg"  # Or your chosen style image

if os.path.exists(style_image_path):
    style_img = Image.open(style_image_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(style_img)
    plt.title("Style Image", fontsize=14, fontweight="bold")
    plt.axis("off")
    plt.show()
else:
    print(f"Style image not found at {style_image_path}")
    print("Please upload a style image or update the path.")

## 3. Training Configuration

Set up training parameters optimized for Colab Free Tier.

In [ ]:
# Training configuration
config = {
    "dataset_path": "val2017",  # Path to COCO images
    "style_image": "picasso_selfportrait.jpg",  # Style image path
    "epochs": 3,  # Number of training epochs
    "batch_size": 4,  # Batch size (adjust based on GPU memory)
    "image_size": 256,  # Image resolution
    "lr": 1e-3,  # Learning rate
    "content_weight": 1.0,  # Content loss weight
    "style_weight": 10.0,  # Style loss weight
    "tv_weight": 1e-6,  # Total variation loss weight
    "subset_size": 2000,  # Use 2000 images from COCO
    "checkpoint_dir": "models/checkpoints",
}

print("Training Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 4. Training the Model

Train the Fast Neural Style Transfer model.

In [ ]:
# Add src to path
import sys

sys.path.append("src")

import os
import time
import torch
import torch.optim as optim
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

from transformer import TransformerNet
from vgg_loss import StyleTransferLoss
from datasets import create_dataloader, get_style_image_transform

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}\n")

# Create output directory
os.makedirs(config["checkpoint_dir"], exist_ok=True)

# Initialize model
print("Initializing transformer network...")
transformer = TransformerNet().to(device)
num_params = sum(p.numel() for p in transformer.parameters())
print(f"Model parameters: {num_params:,}\n")

# Initialize loss function
print("Initializing loss function with VGG19...")
loss_fn = StyleTransferLoss(
    content_weight=config["content_weight"],
    style_weight=config["style_weight"],
    tv_weight=config["tv_weight"],
).to(device)

# Initialize optimizer
optimizer = optim.Adam(transformer.parameters(), lr=config["lr"])
print(f"Optimizer: Adam (lr={config['lr']})\n")

In [ ]:
# Load and pre-process style image
print(f"Loading style image: {config['style_image']}")
style_transform = get_style_image_transform(config["image_size"])
style_image = Image.open(config["style_image"]).convert("RGB")
style_tensor = style_transform(style_image).unsqueeze(0).to(device)

# Pre-compute style Gram matrices
print("Computing style features...")
style_grams = loss_fn.extract_style_gram(style_tensor)
print("Style features computed!\n")

In [ ]:
# Create dataloader
print(f"Loading dataset from: {config['dataset_path']}")
dataloader = create_dataloader(
    config["dataset_path"],
    batch_size=config["batch_size"],
    image_size=config["image_size"],
    subset_size=config["subset_size"],
    shuffle=True,
    num_workers=2,
)
print(f"Dataloader ready: {len(dataloader)} batches per epoch\n")

In [ ]:
# Training loop
print("=" * 60)
print(f"Starting Training: {config['epochs']} epochs")
print("=" * 60)

loss_history = {"total": [], "content": [], "style": [], "tv": []}

for epoch in range(config["epochs"]):
    transformer.train()
    epoch_losses = {"total": 0, "content": 0, "style": 0, "tv": 0}
    num_batches = 0

    epoch_start_time = time.time()

    # Progress bar
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{config['epochs']}")

    for batch_idx, content_batch in enumerate(pbar):
        content_batch = content_batch.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        generated_batch = transformer(content_batch)

        # Compute loss
        losses = loss_fn(
            generated_batch, content_batch, style_features_gram=style_grams
        )

        # Backward pass
        losses["total"].backward()
        optimizer.step()

        # Accumulate losses
        epoch_losses["total"] += losses["total"].item()
        epoch_losses["content"] += losses["content"]
        epoch_losses["style"] += losses["style"]
        epoch_losses["tv"] += losses["tv"]
        num_batches += 1

        # Update progress bar
        pbar.set_postfix(
            {
                "loss": f"{losses['total'].item():.2f}",
                "content": f"{losses['content']:.2f}",
                "style": f"{losses['style']:.2f}",
            }
        )

    # Calculate average losses
    avg_losses = {k: v / num_batches for k, v in epoch_losses.items()}

    # Store losses
    for key in loss_history:
        loss_history[key].append(avg_losses[key])

    epoch_time = time.time() - epoch_start_time

    # Print epoch summary
    print(f"\nEpoch {epoch+1}/{config['epochs']} - Time: {epoch_time:.2f}s")
    print(f"  Avg Total Loss:   {avg_losses['total']:.4f}")
    print(f"  Avg Content Loss: {avg_losses['content']:.4f}")
    print(f"  Avg Style Loss:   {avg_losses['style']:.4f}")
    print(f"  Avg TV Loss:      {avg_losses['tv']:.6f}\n")

    # Save checkpoint
    checkpoint_path = os.path.join(
        config["checkpoint_dir"], f"checkpoint_epoch_{epoch+1}.pth"
    )
    torch.save(transformer.state_dict(), checkpoint_path)
    print(f"✓ Checkpoint saved: {checkpoint_path}\n")

# Save final model
final_model_path = os.path.join(config["checkpoint_dir"], "final_model.pth")
torch.save(transformer.state_dict(), final_model_path)

print("=" * 60)
print("Training Complete!")
print(f"Final model saved: {final_model_path}")
print("=" * 60)

## 5. Visualize Training Progress

Plot the loss curves to see how the model learned.

In [ ]:
# Plot loss curves
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Total loss
axes[0, 0].plot(loss_history["total"], "b-", linewidth=2, marker="o")
axes[0, 0].set_title("Total Loss", fontsize=14, fontweight="bold")
axes[0, 0].set_xlabel("Epoch")
axes[0, 0].set_ylabel("Loss")
axes[0, 0].grid(True, alpha=0.3)

# Content loss
axes[0, 1].plot(loss_history["content"], "g-", linewidth=2, marker="o")
axes[0, 1].set_title("Content Loss", fontsize=14, fontweight="bold")
axes[0, 1].set_xlabel("Epoch")
axes[0, 1].set_ylabel("Loss")
axes[0, 1].grid(True, alpha=0.3)

# Style loss
axes[1, 0].plot(loss_history["style"], "r-", linewidth=2, marker="o")
axes[1, 0].set_title("Style Loss", fontsize=14, fontweight="bold")
axes[1, 0].set_xlabel("Epoch")
axes[1, 0].set_ylabel("Loss")
axes[1, 0].grid(True, alpha=0.3)

# TV loss
axes[1, 1].plot(loss_history["tv"], "m-", linewidth=2, marker="o")
axes[1, 1].set_title("Total Variation Loss", fontsize=14, fontweight="bold")
axes[1, 1].set_xlabel("Epoch")
axes[1, 1].set_ylabel("Loss")
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
loss_plot_path = os.path.join(config["checkpoint_dir"], "loss_plot.png")
plt.savefig(loss_plot_path, dpi=150, bbox_inches="tight")
print(f"Loss plot saved: {loss_plot_path}")
plt.show()

## 6. Inference - Stylize Test Images

Use the trained model to stylize new images.

In [ ]:
# Load the trained model
from inference import load_model, stylize_image
from datasets import tensor_to_image

# Load model
model = load_model(final_model_path, device=device)
print("Model loaded for inference!")

In [ ]:
# Test on a sample image
test_image_path = "japanese_garden.jpg"  # Or upload your own

if os.path.exists(test_image_path):
    output = stylize_image(
        model,
        test_image_path,
        output_path="stylized_output.jpg",
        device=device,
        display=True,
    )
else:
    print(f"Test image not found: {test_image_path}")
    print("Please upload an image or use an image from COCO dataset.")

In [ ]:
# Test on multiple sample images from COCO
import random
from datasets import get_test_image_transform

# Get random images from dataset
coco_images = [
    os.path.join(config["dataset_path"], f)
    for f in os.listdir(config["dataset_path"])
    if f.endswith(".jpg")
][
    :10
]  # Get first 10

# Select 3 random images
sample_images = random.sample(coco_images, min(3, len(coco_images)))

fig, axes = plt.subplots(len(sample_images), 2, figsize=(12, 4 * len(sample_images)))

for idx, img_path in enumerate(sample_images):
    # Load content image
    content_img = Image.open(img_path).convert("RGB")

    # Stylize
    output = stylize_image(model, img_path, device=device, display=False)

    # Display
    if len(sample_images) == 1:
        axes[0].imshow(content_img)
        axes[0].set_title("Original", fontsize=12, fontweight="bold")
        axes[0].axis("off")

        axes[1].imshow(output)
        axes[1].set_title("Stylized", fontsize=12, fontweight="bold")
        axes[1].axis("off")
    else:
        axes[idx, 0].imshow(content_img)
        axes[idx, 0].set_title(f"Original {idx+1}", fontsize=12, fontweight="bold")
        axes[idx, 0].axis("off")

        axes[idx, 1].imshow(output)
        axes[idx, 1].set_title(f"Stylized {idx+1}", fontsize=12, fontweight="bold")
        axes[idx, 1].axis("off")

plt.tight_layout()
plt.savefig("sample_results.png", dpi=150, bbox_inches="tight")
plt.show()

print("\nSample results saved to: sample_results.png")

## 7. Summary and Results

This notebook demonstrated:

1. ✓ Training a Fast Neural Style Transfer model on COCO dataset
2. ✓ Using VGG19 for content and style loss computation
3. ✓ Optimizing for Colab Free Tier (256×256, 2-3 epochs, batch size 4)
4. ✓ Visualizing training progress with loss plots
5. ✓ Running inference on test images

### Key Findings:

- Model successfully learned to apply artistic style to arbitrary images
- Content and style losses decreased over epochs
- Training time: ~X minutes per epoch on Colab GPU
- Inference time: ~X seconds per image

### Next Steps:

- Experiment with different style images
- Adjust loss weights for different stylization effects
- Train for more epochs for better quality
- Try higher resolution images (512×512)

---

**CSE 311 Artificial Intelligence Project**  
*Real-Time Artistic Image Stylization Using Deep Neural Networks*